In [1]:
import mysql.connector as mysql
import pyodbc as pdbo
from pandas import read_csv
import pandas as pd
import numpy as np
# Load libraries
import pandas as pd
#import the library for splitting text
import re
from datetime import datetime
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns; sns.set()
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import math

In [2]:
server = 'x.database.windows.net'
database = 'x'
username = 'x'
password = '{x}'   
driver= '{ODBC Driver 17 for SQL Server}'
#Import data from specific location 
# if overall grading pls check order by only nid quantityperday and select only have itemcode, itemname and quantityperday
# if itemtype grading pls check order by nid add itemtype infront of quantityperday and select nid to add itemtype
# if groupname grading pls check order by nid add groupname infront of quantityperday and select nid to add groupname
cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
query = ("SELECT SPD.Date, SPD.SalesNo, ItemCode,Qty FROM SalesPerDay SPD, Sales S WHERE (S.Location = 'Zugo Mart Sri Petaling' OR S.Location = 'Zugo Mart Changkat' OR S.Location = 'Zugo Mart Kota Damansara') AND S.SalesNo = SPD.SalesNo AND SPD.Date >= '2022-04-01' AND SPD.Date < '2022-07-01' ORDER BY SPD.Date, SPD.SalesNo")
spd = pd.read_sql(query, cnxn)

In [3]:
basket = (spd.groupby(['SalesNo','ItemCode'])['Qty'].sum().unstack().reset_index().fillna(0).set_index('SalesNo'))
basket

ItemCode,01231233,01231932,01232135,01239730,013256110515,013256290514,013256320501,013256510414,013256831502,028400017015,...,ZUGO1097,ZUGO1098,ZUGO1099,ZUGO1102,ZUGO1103,ZUGO1104,ZUGO1105,ZUGO1130,ZUGO1131,ZUGO1132
SalesNo,,,,,,,,,,,,,,,,,,,,,
3071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36020264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36020265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36020266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36024679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36024680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36024681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
def encoder(x):
    if(x <= 0):
        return 0
    else:
        return 1
basket = basket.applymap(encoder)
basket.head()

ItemCode,01231233,01231932,01232135,01239730,013256110515,013256290514,013256320501,013256510414,013256831502,028400017015,...,ZUGO1097,ZUGO1098,ZUGO1099,ZUGO1102,ZUGO1103,ZUGO1104,ZUGO1105,ZUGO1130,ZUGO1131,ZUGO1132
SalesNo,,,,,,,,,,,,,,,,,,,,,
3071,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3072,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36020264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36020265,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36020266,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
freq_items = fpgrowth(basket, min_support=0.00001, use_colnames=True, verbose=0, max_len = 2)
freq_items


,support,itemsets
0,0.000458,(ZUGO1007)
1,0.000229,(4902179021403)
2,0.000229,(9400563455612)
3,0.000229,(8854761951383)
4,0.002289,(8801100129729)
...,...,...
11389,0.000229,"(7622210816917, ZUGO1035)"
11390,0.000229,"(7622210816917, ZUGO1095)"
11391,0.000229,"(VEG10029, VEG10027)"
11392,0.000229,"(VEG10042, VEG10027)"


In [6]:
rules = association_rules(freq_items, metric="lift", min_threshold=0)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ZUGO1018),(ZUGO1007),0.030442,0.000458,0.000229,0.007519,16.424812,0.000215,1.007115
1,(ZUGO1007),(ZUGO1018),0.000458,0.030442,0.000229,0.500000,16.424812,0.000215,1.939117
2,(ZUGO1017),(ZUGO1007),0.032959,0.000458,0.000229,0.006944,15.170139,0.000214,1.006532
3,(ZUGO1007),(ZUGO1017),0.000458,0.032959,0.000229,0.500000,15.170139,0.000214,1.934081
4,(4902179021403),(ZUGO1007),0.000229,0.000458,0.000229,1.000000,2184.500000,0.000229,inf
...,...,...,...,...,...,...,...,...,...
19861,(VEG10027),(VEG10029),0.000229,0.000458,0.000229,1.000000,2184.500000,0.000229,inf
19862,(VEG10042),(VEG10027),0.002518,0.000229,0.000229,0.090909,397.181818,0.000228,1.099748
19863,(VEG10027),(VEG10042),0.000229,0.002518,0.000229,1.000000,397.181818,0.000228,inf
19864,(VEG10044),(VEG10027),0.008927,0.000229,0.000229,0.025641,112.025641,0.000227,1.026081


In [7]:

rules['IR'] = abs(rules['antecedent support']-rules['consequent support'])/(rules['antecedent support']+rules['consequent support']-rules['support'])
rules['cosine'] = rules['support']/np.sqrt(rules['antecedent support']*rules['consequent support'])


In [8]:
rules['Added_Value'] = rules['confidence'] - rules['consequent support']
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,IR,cosine,Added_Value
0,(ZUGO1018),(ZUGO1007),0.030442,0.000458,0.000229,0.007519,16.424812,0.000215,1.007115,0.977612,0.061314,0.007061
1,(ZUGO1007),(ZUGO1018),0.000458,0.030442,0.000229,0.500000,16.424812,0.000215,1.939117,0.977612,0.061314,0.469558
2,(ZUGO1017),(ZUGO1007),0.032959,0.000458,0.000229,0.006944,15.170139,0.000214,1.006532,0.979310,0.058926,0.006487
3,(ZUGO1007),(ZUGO1017),0.000458,0.032959,0.000229,0.500000,15.170139,0.000214,1.934081,0.979310,0.058926,0.467041
4,(4902179021403),(ZUGO1007),0.000229,0.000458,0.000229,1.000000,2184.500000,0.000229,inf,0.500000,0.707107,0.999542
...,...,...,...,...,...,...,...,...,...,...,...,...
19861,(VEG10027),(VEG10029),0.000229,0.000458,0.000229,1.000000,2184.500000,0.000229,inf,0.500000,0.707107,0.999542
19862,(VEG10042),(VEG10027),0.002518,0.000229,0.000229,0.090909,397.181818,0.000228,1.099748,0.909091,0.301511,0.090680
19863,(VEG10027),(VEG10042),0.000229,0.002518,0.000229,1.000000,397.181818,0.000228,inf,0.909091,0.301511,0.997482
19864,(VEG10044),(VEG10027),0.008927,0.000229,0.000229,0.025641,112.025641,0.000227,1.026081,0.974359,0.160128,0.025412


In [9]:
rules = rules.sort_values(by=['Added_Value'], ascending = False)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,IR,cosine,Added_Value
19451,(6928104230991),(6928104231004),0.000229,0.000229,0.000229,1.000000,4369.000000,0.000229,inf,0.000000,1.000000,0.999771
5413,(4712883830082),(4712883830099),0.000229,0.000229,0.000229,1.000000,4369.000000,0.000229,inf,0.000000,1.000000,0.999771
4602,(648436101481),(9556184666101),0.000229,0.000229,0.000229,1.000000,4369.000000,0.000229,inf,0.000000,1.000000,0.999771
15513,(8801052012469),(6901844524117),0.000229,0.000229,0.000229,1.000000,4369.000000,0.000229,inf,0.000000,1.000000,0.999771
16535,(8992760223015),(7622210708434),0.000229,0.000229,0.000229,1.000000,4369.000000,0.000229,inf,0.000000,1.000000,0.999771
...,...,...,...,...,...,...,...,...,...,...,...,...
1123,(9555143600019),(ZUGO1011),0.017395,0.059281,0.000229,0.013158,0.221957,-0.000802,0.953262,0.547904,0.007128,-0.046123
1784,(9556135421506),(ZUGO1011),0.019913,0.059281,0.000229,0.011494,0.193893,-0.000952,0.951657,0.498551,0.006662,-0.047787
1119,(9555143600019),(ZUGO1032),0.017395,0.065003,0.000229,0.013158,0.202418,-0.000902,0.947463,0.579387,0.006807,-0.051846
1778,(9556135421506),(ZUGO1032),0.019913,0.065003,0.000229,0.011494,0.176825,-0.001066,0.945869,0.532432,0.006362,-0.053509


In [10]:
df = pd.DataFrame(rules, columns = ['antecedents','consequents','antecedent support','consequent support','support','confidence','lift','conviction','Added_Value','leverage'])
df = (df.explode('antecedents')) 
df = (df.explode('consequents'))
df.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,conviction,Added_Value,leverage
19451,6928104230991,6928104231004,0.000229,0.000229,0.000229,1.0,4369.0,inf,0.999771,0.000229
5413,4712883830082,4712883830099,0.000229,0.000229,0.000229,1.0,4369.0,inf,0.999771,0.000229
4602,648436101481,9556184666101,0.000229,0.000229,0.000229,1.0,4369.0,inf,0.999771,0.000229
15513,8801052012469,6901844524117,0.000229,0.000229,0.000229,1.0,4369.0,inf,0.999771,0.000229
16535,8992760223015,7622210708434,0.000229,0.000229,0.000229,1.0,4369.0,inf,0.999771,0.000229


In [12]:
basket2 = basket.T
basket2['ItemCode']= basket2.index.values

In [13]:
df2 = df[(df.lift>=1)]
df2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,conviction,Added_Value,leverage
12903,ZUGO1008,9556072080057,0.000458,0.001144,0.000229,0.500000,436.900000,1.997711,0.498856,0.000228
12904,ZUGO1008,RGE10029,0.000458,0.007324,0.000229,0.500000,68.265625,1.985351,0.492676,0.000226
2981,ZUGO1013,ZUGO1018,0.016709,0.030442,0.002747,0.164384,5.399938,1.160291,0.133942,0.002238
2949,ZUGO1009,ZUGO1032,0.030900,0.065003,0.004807,0.155556,2.393036,1.107233,0.090552,0.002798
2983,ZUGO1013,ZUGO1017,0.016709,0.032959,0.002060,0.123288,3.740582,1.103031,0.090328,0.001509
...,...,...,...,...,...,...,...,...,...,...
921,ZUGO1011,6921571193554,0.059281,0.003662,0.000229,0.003861,1.054295,1.000200,0.000199,0.000012
3944,ZUGO1011,RGE10003,0.059281,0.003662,0.000229,0.003861,1.054295,1.000200,0.000199,0.000012
4709,ZUGO1009,RGE10029,0.030900,0.007324,0.000229,0.007407,1.011343,1.000084,0.000083,0.000003
5875,ZUGO1009,BF10004,0.030900,0.007324,0.000229,0.007407,1.011343,1.000084,0.000083,0.000003


In [14]:
df2['ProbabilitiesIndex'] = df2['lift']*df2['confidence']*df2['support']

C:\Users\JACK\AppData\Local\Temp/ipykernel_25900/1716442595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ProbabilitiesIndex'] = df2['lift']*df2['confidence']*df2['support']


In [15]:
df2['ConfidenceOccurs'] = df2['confidence']*df2['support']

C:\Users\JACK\AppData\Local\Temp/ipykernel_25900/3545573929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ConfidenceOccurs'] = df2['confidence']*df2['support']


In [16]:
df2.sort_values(by=['ConfidenceOccurs'],ascending= [False])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,conviction,Added_Value,leverage,ProbabilitiesIndex,ConfidenceOccurs
956,ZUGO1011,ZUGO1032,0.059281,0.065003,0.006867,0.115830,1.781908,1.057485,0.050827,0.003013,0.001417,7.953544e-04
2949,ZUGO1009,ZUGO1032,0.030900,0.065003,0.004807,0.155556,2.393036,1.107233,0.090552,0.002798,0.001789,7.476921e-04
2981,ZUGO1013,ZUGO1018,0.016709,0.030442,0.002747,0.164384,5.399938,1.160291,0.133942,0.002238,0.002438,4.514998e-04
2977,ZUGO1009,ZUGO1018,0.030900,0.030442,0.003662,0.118519,3.893289,1.099919,0.088077,0.002722,0.001690,4.340344e-04
2895,ZUGO1011,ZUGO1017,0.059281,0.032959,0.004578,0.077220,2.342879,1.047964,0.044261,0.002624,0.000828,3.534909e-04
...,...,...,...,...,...,...,...,...,...,...,...,...
7164,ZUGO1011,4514603362619,0.059281,0.000916,0.000229,0.003861,4.217181,1.002957,0.002945,0.000175,0.000004,8.837271e-07
11444,ZUGO1011,9556404057030,0.059281,0.000916,0.000229,0.003861,4.217181,1.002957,0.002945,0.000175,0.000004,8.837271e-07
5773,ZUGO1011,9556127202014,0.059281,0.000916,0.000229,0.003861,4.217181,1.002957,0.002945,0.000175,0.000004,8.837271e-07
17275,ZUGO1011,8935106509305,0.059281,0.000916,0.000229,0.003861,4.217181,1.002957,0.002945,0.000175,0.000004,8.837271e-07


In [17]:
df3 = pd.DataFrame(df2.groupby(['antecedents']).count().consequents)

In [18]:
df3['antecedents'] = df3.index.values

In [19]:
df2['Affected_Items_Count'] = df2['antecedents'].map(df3.set_index('antecedents')['consequents'])
df2

C:\Users\JACK\AppData\Local\Temp/ipykernel_25900/833388329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Affected_Items_Count'] = df2['antecedents'].map(df3.set_index('antecedents')['consequents'])


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,conviction,Added_Value,leverage,ProbabilitiesIndex,ConfidenceOccurs,Affected_Items_Count
12903,ZUGO1008,9556072080057,0.000458,0.001144,0.000229,0.500000,436.900000,1.997711,0.498856,0.000228,5.000000e-02,1.144427e-04,2
12904,ZUGO1008,RGE10029,0.000458,0.007324,0.000229,0.500000,68.265625,1.985351,0.492676,0.000226,7.812500e-03,1.144427e-04,2
2981,ZUGO1013,ZUGO1018,0.016709,0.030442,0.002747,0.164384,5.399938,1.160291,0.133942,0.002238,2.438071e-03,4.514998e-04,79
2949,ZUGO1009,ZUGO1032,0.030900,0.065003,0.004807,0.155556,2.393036,1.107233,0.090552,0.002798,1.789254e-03,7.476921e-04,140
2983,ZUGO1013,ZUGO1017,0.016709,0.032959,0.002060,0.123288,3.740582,1.103031,0.090328,0.001509,9.499906e-04,2.539687e-04,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...
921,ZUGO1011,6921571193554,0.059281,0.003662,0.000229,0.003861,1.054295,1.000200,0.000199,0.000012,9.317094e-07,8.837271e-07,177
3944,ZUGO1011,RGE10003,0.059281,0.003662,0.000229,0.003861,1.054295,1.000200,0.000199,0.000012,9.317094e-07,8.837271e-07,177
4709,ZUGO1009,RGE10029,0.030900,0.007324,0.000229,0.007407,1.011343,1.000084,0.000083,0.000003,1.714678e-06,1.695447e-06,140
5875,ZUGO1009,BF10004,0.030900,0.007324,0.000229,0.007407,1.011343,1.000084,0.000083,0.000003,1.714678e-06,1.695447e-06,140


In [20]:
df2['Probabilities&Affected_Items_Count'] = df2['ProbabilitiesIndex']*df2['Affected_Items_Count']
df2['ConfidenceOccurs&Affected_Items_Count'] = df2['ConfidenceOccurs']*df2['Affected_Items_Count']

C:\Users\JACK\AppData\Local\Temp/ipykernel_25900/3615027409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Probabilities&Affected_Items_Count'] = df2['ProbabilitiesIndex']*df2['Affected_Items_Count']
C:\Users\JACK\AppData\Local\Temp/ipykernel_25900/3615027409.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ConfidenceOccurs&Affected_Items_Count'] = df2['ConfidenceOccurs']*df2['Affected_Items_Count']


In [21]:
df2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,conviction,Added_Value,leverage,ProbabilitiesIndex,ConfidenceOccurs,Affected_Items_Count,Probabilities&Affected_Items_Count,ConfidenceOccurs&Affected_Items_Count
12903,ZUGO1008,9556072080057,0.000458,0.001144,0.000229,0.500000,436.900000,1.997711,0.498856,0.000228,5.000000e-02,1.144427e-04,2,0.100000,0.000229
12904,ZUGO1008,RGE10029,0.000458,0.007324,0.000229,0.500000,68.265625,1.985351,0.492676,0.000226,7.812500e-03,1.144427e-04,2,0.015625,0.000229
2981,ZUGO1013,ZUGO1018,0.016709,0.030442,0.002747,0.164384,5.399938,1.160291,0.133942,0.002238,2.438071e-03,4.514998e-04,79,0.192608,0.035668
2949,ZUGO1009,ZUGO1032,0.030900,0.065003,0.004807,0.155556,2.393036,1.107233,0.090552,0.002798,1.789254e-03,7.476921e-04,140,0.250496,0.104677
2983,ZUGO1013,ZUGO1017,0.016709,0.032959,0.002060,0.123288,3.740582,1.103031,0.090328,0.001509,9.499906e-04,2.539687e-04,79,0.075049,0.020064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921,ZUGO1011,6921571193554,0.059281,0.003662,0.000229,0.003861,1.054295,1.000200,0.000199,0.000012,9.317094e-07,8.837271e-07,177,0.000165,0.000156
3944,ZUGO1011,RGE10003,0.059281,0.003662,0.000229,0.003861,1.054295,1.000200,0.000199,0.000012,9.317094e-07,8.837271e-07,177,0.000165,0.000156
4709,ZUGO1009,RGE10029,0.030900,0.007324,0.000229,0.007407,1.011343,1.000084,0.000083,0.000003,1.714678e-06,1.695447e-06,140,0.000240,0.000237
5875,ZUGO1009,BF10004,0.030900,0.007324,0.000229,0.007407,1.011343,1.000084,0.000083,0.000003,1.714678e-06,1.695447e-06,140,0.000240,0.000237


In [22]:
cnxn = pdbo.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
query = ("SELECT ItemCode AS antecedents, ItemName FROM Items")
itemlist = pd.read_sql(query, cnxn)

In [23]:
df2['ItemNameant'] = df2['antecedents'].map(itemlist.set_index('antecedents')['ItemName'])
df2

C:\Users\JACK\AppData\Local\Temp/ipykernel_25900/2974977597.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ItemNameant'] = df2['antecedents'].map(itemlist.set_index('antecedents')['ItemName'])


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,conviction,Added_Value,leverage,ProbabilitiesIndex,ConfidenceOccurs,Affected_Items_Count,Probabilities&Affected_Items_Count,ConfidenceOccurs&Affected_Items_Count,ItemNameant
12903,ZUGO1008,9556072080057,0.000458,0.001144,0.000229,0.500000,436.900000,1.997711,0.498856,0.000228,5.000000e-02,1.144427e-04,2,0.100000,0.000229,ZUGO ESPRESSO
12904,ZUGO1008,RGE10029,0.000458,0.007324,0.000229,0.500000,68.265625,1.985351,0.492676,0.000226,7.812500e-03,1.144427e-04,2,0.015625,0.000229,ZUGO ESPRESSO
2981,ZUGO1013,ZUGO1018,0.016709,0.030442,0.002747,0.164384,5.399938,1.160291,0.133942,0.002238,2.438071e-03,4.514998e-04,79,0.192608,0.035668,ZUGO HOT AMERICANO
2949,ZUGO1009,ZUGO1032,0.030900,0.065003,0.004807,0.155556,2.393036,1.107233,0.090552,0.002798,1.789254e-03,7.476921e-04,140,0.250496,0.104677,ZUGO HOT LATTE
2983,ZUGO1013,ZUGO1017,0.016709,0.032959,0.002060,0.123288,3.740582,1.103031,0.090328,0.001509,9.499906e-04,2.539687e-04,79,0.075049,0.020064,ZUGO HOT AMERICANO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921,ZUGO1011,6921571193554,0.059281,0.003662,0.000229,0.003861,1.054295,1.000200,0.000199,0.000012,9.317094e-07,8.837271e-07,177,0.000165,0.000156,ZUGO ICED LATTE
3944,ZUGO1011,RGE10003,0.059281,0.003662,0.000229,0.003861,1.054295,1.000200,0.000199,0.000012,9.317094e-07,8.837271e-07,177,0.000165,0.000156,ZUGO ICED LATTE
4709,ZUGO1009,RGE10029,0.030900,0.007324,0.000229,0.007407,1.011343,1.000084,0.000083,0.000003,1.714678e-06,1.695447e-06,140,0.000240,0.000237,ZUGO HOT LATTE
5875,ZUGO1009,BF10004,0.030900,0.007324,0.000229,0.007407,1.011343,1.000084,0.000083,0.000003,1.714678e-06,1.695447e-06,140,0.000240,0.000237,ZUGO HOT LATTE


In [24]:
df2['ItemNamecon'] = df2['consequents'].map(itemlist.set_index('antecedents')['ItemName'])
df2

C:\Users\JACK\AppData\Local\Temp/ipykernel_25900/3269619986.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ItemNamecon'] = df2['consequents'].map(itemlist.set_index('antecedents')['ItemName'])


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,conviction,Added_Value,leverage,ProbabilitiesIndex,ConfidenceOccurs,Affected_Items_Count,Probabilities&Affected_Items_Count,ConfidenceOccurs&Affected_Items_Count,ItemNameant,ItemNamecon
12903,ZUGO1008,9556072080057,0.000458,0.001144,0.000229,0.500000,436.900000,1.997711,0.498856,0.000228,5.000000e-02,1.144427e-04,2,0.100000,0.000229,ZUGO ESPRESSO,TWISTIES CHEEZELS CHEESE 60G
12904,ZUGO1008,RGE10029,0.000458,0.007324,0.000229,0.500000,68.265625,1.985351,0.492676,0.000226,7.812500e-03,1.144427e-04,2,0.015625,0.000229,ZUGO ESPRESSO,FRIED CHEESE TOFU
2981,ZUGO1013,ZUGO1018,0.016709,0.030442,0.002747,0.164384,5.399938,1.160291,0.133942,0.002238,2.438071e-03,4.514998e-04,79,0.192608,0.035668,ZUGO HOT AMERICANO,ZUGO BIHUN GORENG
2949,ZUGO1009,ZUGO1032,0.030900,0.065003,0.004807,0.155556,2.393036,1.107233,0.090552,0.002798,1.789254e-03,7.476921e-04,140,0.250496,0.104677,ZUGO HOT LATTE,ZUGO HERBAL TEA EGG 2PCS
2983,ZUGO1013,ZUGO1017,0.016709,0.032959,0.002060,0.123288,3.740582,1.103031,0.090328,0.001509,9.499906e-04,2.539687e-04,79,0.075049,0.020064,ZUGO HOT AMERICANO,ZUGO NASI LEMAK BUNGKUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921,ZUGO1011,6921571193554,0.059281,0.003662,0.000229,0.003861,1.054295,1.000200,0.000199,0.000012,9.317094e-07,8.837271e-07,177,0.000165,0.000156,ZUGO ICED LATTE,HAIYI KISS DELICIAS SEA SALT PASSION FRUIT CAN...
3944,ZUGO1011,RGE10003,0.059281,0.003662,0.000229,0.003861,1.054295,1.000200,0.000199,0.000012,9.317094e-07,8.837271e-07,177,0.000165,0.000156,ZUGO ICED LATTE,WHITE FISHBALL
4709,ZUGO1009,RGE10029,0.030900,0.007324,0.000229,0.007407,1.011343,1.000084,0.000083,0.000003,1.714678e-06,1.695447e-06,140,0.000240,0.000237,ZUGO HOT LATTE,FRIED CHEESE TOFU
5875,ZUGO1009,BF10004,0.030900,0.007324,0.000229,0.007407,1.011343,1.000084,0.000083,0.000003,1.714678e-06,1.695447e-06,140,0.000240,0.000237,ZUGO HOT LATTE,MEXICO COFFEE BUN


In [ ]:
df2 = df2[['antecedents','consequents','ItemNameant','ItemNamecon','support','confidence','conviction','ConfidenceOccurs','Affected_Items_Count','Added_Value','leverage']]
df2

In [25]:
df2.to_excel("D:\leadingItemsJune.xlsx")